# Custom indicator example

In [11]:
import logging
import yfinance as yf

from mplchart.chart import Chart

from mplchart.library import get_series, calc_ema
from mplchart.primitives import Candlesticks, OHLC, Volume

from dataclasses import dataclass

logging.basicConfig(level=logging.INFO)

In [12]:
ticker = "AAPL"
prices = yf.Ticker(ticker).history(period="5y")
prices

In [16]:
from mplchart.utils import series_xy


class LoggedAccess:
    def __set_name__(self, owner, name):
        self.name = name

    def __get__(self, obj, objtype=None):
        value = getattr(obj, self.private_name)
        logging.info('Accessing %r giving %r', self.public_name, value)
        return value

    def __set__(self, obj, value):
        logging.info('Updating %r to %r', self.public_name, value)
        setattr(obj, self.private_name, value)

@dataclass
class DEMA:
    """Double Exponential Moving Average"""

    period: int = 20

    same_scale = True
    # same_scale is an optional class attribute
    # to specify that the indicator can be drawn
    # on the same axes as the previous indicator

    def __call__(self, prices):
        series = get_series(prices)
        ema1 = calc_ema(series, self.period)
        ema2 = calc_ema(ema1, self.period)
        return 2 * ema1 - ema2
    
    def plot_result(self, data, chart, ax=None):
        logging.info("plot_result")
        
        if ax is None:
            ax = chart.get_axes("samex")

        label = repr(self)
        xv, yv = series_xy(data)

        ax.plot(xv, yv, label=label)


            

In [17]:
res = DEMA(20)(prices)
res.tail()

In [18]:
max_bars = 250

indicators = [
    OHLC(),
    DEMA(20),
]

chart = Chart(title=ticker, max_bars=max_bars)
chart.plot(prices, indicators)